In [1]:
from __future__ import division, print_function
import importlib, math, os, shutil, textwrap, warnings
from os.path import join, basename
import numpy as np
import caffe
from caffe.proto import caffe_pb2
from google import protobuf
from caffe import layers as L
from caffe import params as P
from experimenter.utilities import num_lines, get_latest_snapshot, mkdir_p, num_lines
from experimenter.nets import caffenet, write_net
from experimenter.nets.caffenet import create_param, default_layer_names
%load_ext autoreload
%autoreload 2

In [2]:
group_name = 'experiment3'
group_desc = textwrap.dedent('''0.00001 lr, just to test decaying
''')

from experiment import generate, datadir, basedir, train_batch_size, val_batch_size, test_batch_size, num_classes

test_file = datadir + 'test.txt'

In [3]:
def filter_mults2param(mults):
    return [ create_param(flr_mult=mult) for mult in mults ]
def rename2names(rename_indices):
    names = default_layer_names[:]
    for idx in rename_indices:
        names[idx] = names[idx] + '_randominit'
    return names

experiment_names = [
    'shallowtune',
    'deepertune',
    'deeptune',
    'shallowfreeze',
    'deeperfreeze',
    'deepfreeze'
]
# order of learning rates: input layer to output layer
filter_mults = [
    [1] * 7 + [5],
    [1] * 5 + [5] * 3,
    [1] * 3 + [5] * 5,
    [0] * 7 + [5],
    [0] * 5 + [5] * 3,
    [0] * 3 + [5] * 5,
]
experiment_params = [ filter_mults2param(mult_list) for mult_list in filter_mults ]

# if you change name of layer, caffe will just randomly initialize
# (caffe will only load as many weights as possible, names that don’t match are ignored)
experiment_layer_rename = [
    [7],
    [5,6,7],
    [3,4,5,6,7],
    [7],
    [5,6,7],
    [3,4,5,6,7]
]
experiment_layer_names = map(rename2names, experiment_layer_rename)

num_test = num_lines(test_file)

group_folder = join(basedir, group_name)
mkdir_p(group_folder)
with open(join(group_folder, 'DESC'), 'w') as f:
    f.write(group_desc + '\n')
shutil.copy(test_file, group_folder)
test_path = join(group_folder, basename(test_file))

for exp in generate(group_name):
    dataset = exp.dataset
    n_train, n_val = dataset.n_train, dataset.n_val
    
    parameter_index = experiment_names.index(exp.name)
    experiment_param = experiment_params[parameter_index]
    experiment_layers = experiment_layer_names[parameter_index]
    
    folder = exp.folder
    mkdir_p(folder)
    if get_latest_snapshot(folder) != None:
        raise RuntimeError('Snapshots still in folder. You might be writing to wrong experiment.')

    # store copies of data lists (they must be absolute paths or paths in the same folder, not .. paths)
    shutil.copy(dataset.train_path, group_folder)
    shutil.copy(dataset.val_path, group_folder)
    train_path = join(group_folder, basename(dataset.train_path))
    val_path = join(group_folder, basename(dataset.val_path))

    #### Training net
    val_batch_size=min(val_batch_size, n_val)
    net = caffenet.build(train=train_path, test=val_path, params=experiment_param,
                   names=experiment_layers, train_batch_size=train_batch_size, test_batch_size=val_batch_size,
                   num_classes=num_classes)
    write_net(net, join(folder,'train_val.prototxt'))

    ### Training solver
    val_iters = int(n_val / val_batch_size)
    iters_per_epoch = n_train / train_batch_size
    iters_per_gamma = None if exp.decay_epochs == 0 else int(exp.decay_epochs * iters_per_epoch)
    # snapshot prefix must have trailing slash
    snapshot_prefix = join(folder, '')
    caffenet.write_solver(join(folder, 'train_val.prototxt'), test_iters=val_iters, average_loss=iters_per_epoch,
                          snapshot_prefix=snapshot_prefix, stepsize=iters_per_gamma, gamma=exp.gamma,
                          base_lr=exp.base_lr, weight_decay=exp.weight_decay,
                          dst=join(folder, 'solver.prototxt'))

    #### Test net
    test_net = caffenet.build(train=None, test=test_path, params=experiment_param,
                        names=experiment_layers, test_batch_size=test_batch_size, num_classes=num_classes)
    write_net(test_net, join(folder, 'test.prototxt'))

    #### Test solver
    test_iters = int(num_test / test_batch_size)
    caffenet.write_solver(join(folder, 'test.prototxt'), test_iters=test_iters, average_loss=1,
                          snapshot_prefix=join(folder, ''), dst=join(folder, 'test_solver.prototxt'))

# NOTE: test_batch_size * test_iter must = total number of test images (validation set)